In [11]:
import numpy as np
import os
import cv2

KNN MODEL

In [12]:
def distance(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(train_X,train_Y,query_point,k=10):
    m=train_X.shape[0]
    vals=[]
    for i in range(m):
        d=distance(train_X[i],query_point)
        vals.append((d,train_Y[i]))
        
    vals=sorted(vals)
    vals=vals[:k]
    vals=np.array(vals)
    new_vals=np.unique(vals[:,1],return_counts=True)
    index=new_vals[1].argmax()
    pred=new_vals[0][index]
    return pred

DATA LOADING/PREPARATION

In [13]:
dataset_path=r"C:\Users\dell\Desktop\projects\Machine Learning\CBMC\Real Time Face Recognition"
face_data=[] #list of list of faces
labels=[]
class_id=0
names={}
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        data_items=np.load(dataset_path+"\\"+fx)
        face_data.append(data_items)
        #create_labels
        target=class_id*np.ones((data_items.shape[0],))
        labels.append(target)
        class_id+=1
        
face_data=np.concatenate(face_data,axis=0) #this is our X_train
labels=np.concatenate(labels,axis=0).reshape((-1,1))   #we want only one column and this is going to be our Y_train
        

TESTING 

In [15]:
cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier(r"C:\Users\dell\Desktop\projects\Machine Learning\Coding Blocks\Real Time Face Recognition\haarcascade_frontalface_default.xml")

while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    #iterating over all the faces
    for face in faces:
        x,y,w,h=face
        
        #we need to get the region of interest that is our face_section
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        
        face_section=cv2.resize(face_section,(100,100))
        
        #predicting the output
        op=knn(face_data,labels,face_section.flatten())
        
        pred_name=names[int(op)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),1)
        
    cv2.imshow("project",frame)       
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
        

In [ ]:
cv2.putText?